## Parsing Ecommerce Data 

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pyodbc

In [2]:
pathname = os.getcwd()
ecom_csv_files = glob.glob(os.path.join(pathname,'*.csv'))

In [3]:
final_Superstore_Df= pd.DataFrame()
for files in ecom_csv_files:
    new_files= files.replace('\\','/')
    Superstore_df = pd.read_csv(new_files,encoding = 'windows-1252')
    Superstore_df.drop_duplicates(keep='first',inplace=True)
    Superstore_df.drop(['Row ID'],axis=1,inplace=True)
    final_Superstore_Df= final_Superstore_Df.append(Superstore_df)

In [4]:
final_Superstore_Df= final_Superstore_Df.reset_index()
final_Superstore_Df.drop(['index'],axis=1,inplace=True)
final_Superstore_Df=final_Superstore_Df.drop_duplicates(keep='last')

In [5]:
customer_df=final_Superstore_Df[['Customer ID','Customer Name']]
customer_df=customer_df.drop_duplicates(keep='first')
customer_df=customer_df.reset_index()
customer_df.drop(['index'],axis=1,inplace=True)

In [6]:
location_df=final_Superstore_Df[['Country','City','State','Postal Code','Region']]
location_df=location_df.drop_duplicates(keep='first')
location_df=location_df.reset_index()
location_df.drop(['index'],axis=1,inplace=True)

In [24]:
product_df=final_Superstore_Df[['Product ID','Product Name','Category','Sub-Category']]
product_df=product_df.drop_duplicates(keep='first')
product_df=product_df.reset_index()
product_df.drop(['index'],axis=1,inplace=True)

In [23]:
order_df=final_Superstore_Df[['Order ID','Order Date','Ship Date','Sales','Quantity','Discount','Profit','Customer ID','Product ID','Postal Code']]
order_df=order_df.drop_duplicates(keep='first')
order_df=order_df.reset_index()
order_df.drop(['index'],axis=1,inplace=True)

In [9]:
customer_df = customer_df.rename(columns={'Customer ID':'Customer_ID','Customer Name':'Customer_Name'})

In [10]:
location_df=location_df.rename(columns={'Postal Code': 'Postal_Code'})

In [27]:
product_df=product_df.rename(columns={'Product ID': 'Product_ID','Product Name': 'Product_Name'})

In [30]:
order_df=order_df.rename(columns={'Order ID': 'Order_ID','Order Date': 'Order_Date','Ship Date':'Ship_Date','Customer ID':'Customer_ID','Product ID':'Product_ID','Postal Code':'Postal_Code'})

In [12]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=DESKTOP-DK61R8P;Database=Ecommerce;Trusted_Connection=yes;')
cursor= conn.cursor()

In [15]:
order_df.head()

,Order_ID,Order_Date,Ship_Date,Sales,Quantity,Discount,Profit,Customer_ID,Product_ID,Postal_Code
0,CA-2016-152156,11/8/2016,11/11/2016,261.9600,2,0.00,41.9136,CG-12520,FUR-BO-10001798,42420
1,CA-2016-152156,11/8/2016,11/11/2016,731.9400,3,0.00,219.5820,CG-12520,FUR-CH-10000454,42420
2,CA-2016-138688,6/12/2016,6/16/2016,14.6200,2,0.00,6.8714,DV-13045,OFF-LA-10000240,90036
3,US-2015-108966,10/11/2015,10/18/2015,957.5775,5,0.45,-383.0310,SO-20335,FUR-TA-10000577,33311
4,US-2015-108966,10/11/2015,10/18/2015,22.3680,2,0.20,2.5164,SO-20335,OFF-ST-10000760,33311


In [14]:
for i,row in customer_df.iterrows():
    cursor.execute("insert into tbl_customer(Customer_ID,Customer_Name)values(?,?)",row.Customer_ID,row.Customer_Name)
    cursor.commit()

DataError: ('22001', '[22001] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]String or binary data would be truncated. (8152) (SQLExecDirectW); [22001] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')

In [18]:
for i,row in location_df.iterrows():
    cursor.execute("insert into tbl_location_df(Country,City,State,Postal_Code,Region)values(?,?,?,?,?)",row.Country,row.City,row.State,row.Postal_Code,row.Region)
    cursor.commit()

In [28]:
for i,row in product_df.iterrows():
    cursor.execute("insert into tbl_product_df(Product_ID,Product_Name,Category,Sub-Category)values(?,?,?,?)",row.Product_ID,row.Product_Name,row.Category,row.Sub-Category)
    cursor.commit()

AttributeError: 'Series' object has no attribute 'Sub'

In [33]:
for i,row in order_df.iterrows():
    cursor.execute("insert into order_df(Order_ID,Order_Date,Ship_Date,Sales,Quantity,Discount,Profit,Customer_ID,Product_ID,Postal_Code)values(?,?,?,?,?,?,?,?,?,?)",row.Order_ID,row.Order_Date,row.Ship_Date,row.Sales,row.Quantity,row.Profit,row.Customer_ID,row.Product_ID,row.Postal_Code)
    cursor.commit()

ProgrammingError: ('The SQL contains 10 parameter markers, but 9 parameters were supplied', 'HY000')

In [19]:
'Series' object has no attribute 'Sub'
location_df.shape

(632, 5)

In [10]:
Superstore_df.shape

(1094, 20)

In [5]:
final_Superstore_Df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [21]:
Superstore_df.drop_duplicates(subset=['Order ID', 'Order Date', 'Order Date', 'Ship Mode'],keep='first',inplace=True)

In [28]:
final_Superstore_Df.shape

(12996, 20)

In [15]:
final_Superstore_Df.shape

(12996, 20)

In [18]:
final_Superstore_Df= final_Superstore_Df.reset_index()

In [19]:
final_Superstore_Df.head()

,index,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,2,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,3,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [25]:
final_Superstore_Df.drop(['index'],axis=1,inplace=True) #droping column
                               

In [26]:
final_Superstore_Df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [27]:
final_Superstore_Df.duplicated().sum()

3003

In [28]:
(~final_Superstore_Df.duplicated()).sum()

9993

In [30]:
final_Superstore_Df.loc[final_Superstore_Df.duplicated(),:].head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
101,CA-2016-158568,8/29/2016,9/2/2016,Standard Class,RB-19465,Rick Bensley,Home Office,United States,Chicago,Illinois,60610,Central,TEC-AC-10001767,Technology,Accessories,SanDisk Ultra 64 GB MicroSDHC Class 10 Memory ...,95.976,3,0.2,-10.7973
3407,US-2014-150119,4/23/2014,4/27/2014,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
8000,CA-2016-129238,1/31/2016,2/4/2016,Standard Class,SC-20050,Sample Company A,Home Office,United States,Los Angeles,California,90045,West,OFF-PA-10002764,Office Supplies,Paper,Easy-staple paper,56.700,5,0.0,27.7830
8001,CA-2017-159688,5/7/2017,5/12/2017,Standard Class,AB-10060,Adam Bellavance,Home Office,United States,Los Angeles,California,90004,West,TEC-AC-10000736,Technology,Accessories,Logitech G600 MMO Gaming Mouse,79.990,1,0.0,28.7964
8002,CA-2016-136126,5/24/2016,5/24/2016,Same Day,EH-14125,Eugene Hildebrand,Home Office,United States,Newport News,Virginia,23602,South,OFF-SU-10000898,Office Supplies,Supplies,Acme Hot Forged Carbon Steel Scissors with Nic...,69.500,5,0.0,20.1550


In [31]:
final_Superstore_Df=final_Superstore_Df.drop_duplicates(keep='last')

In [33]:
final_Superstore_Df.duplicated().sum()

0

In [34]:
final_Superstore_Df.isna().sum()  #to check null values

Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64